In [8]:
import numpy as np
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
max_words = 10000
max_len = 100


In [9]:
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=max_words)
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

print(f"Training data shape: {X_train.shape}")
print(f"Testing data shape: {X_test.shape}")


17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Training data shape: (25000, 100)
Testing data shape: (25000, 100)


In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense

gru_model = Sequential()
gru_model.add(Embedding(input_dim=max_words, output_dim=64, input_length=max_len))
gru_model.add(GRU(128))
gru_model.add(Dense(1, activation='sigmoid'))

gru_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

gru_model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru (GRU)                            │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [11]:
gru_history = gru_model.fit(X_train, y_train, epochs=5, batch_size=128, validation_data=(X_test, y_test))

Epoch 1/5
196/196 ━━━━━━━━━━━━━━━━━━━━ 95s 471ms/step - accuracy: 0.6593 - loss: 0.5939 - val_accuracy: 0.8235 - val_loss: 0.3870
Epoch 2/5
196/196 ━━━━━━━━━━━━━━━━━━━━ 139s 458ms/step - accuracy: 0.8828 - loss: 0.2837 - val_accuracy: 0.8520 - val_loss: 0.3500
Epoch 3/5
196/196 ━━━━━━━━━━━━━━━━━━━━ 143s 461ms/step - accuracy: 0.9179 - loss: 0.2180 - val_accuracy: 0.8433 - val_loss: 0.3578
Epoch 4/5
196/196 ━━━━━━━━━━━━━━━━━━━━ 92s 468ms/step - accuracy: 0.9414 - loss: 0.1627 - val_accuracy: 0.8463 - val_loss: 0.3866
Epoch 5/5
196/196 ━━━━━━━━━━━━━━━━━━━━ 142s 467ms/step - accuracy: 0.9544 - loss: 0.1273 - val_accuracy: 0.8359 - val_loss: 0.4561


In [19]:
tokenizer = tf.keras.preprocessing.text.Tokenizer()
X_train_str = [' '.join(map(str, sequence)) for sequence in X_train]
tokenizer.fit_on_texts(X_train_str)

In [26]:
def generate_text(model, tokenizer, seed_text, max_sequence_len):
    for _ in range(50):  # Generate 50 words
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = model.predict(token_list, verbose=0)
        output_word = ''
        for word, index in tokenizer.word_index.items():
            if index == np.argmax(predicted):
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text

tokenizer = tf.keras.preprocessing.text.Tokenizer()

X_train_str = [' '.join(map(str, sequence)) for sequence in X_train]

tokenizer.fit_on_texts(X_train_str)

seed_text = "The movie was"
generated_text = generate_text(gru_model, tokenizer, seed_text, max_len)
print(f"Generated Text (GRU): {generated_text}")

Generated Text (GRU): The movie was                                                  


In [28]:
max_words = len(tokenizer.word_index) + 1  # Update max_words to cover all unique words + 1 for padding

lstm_model = Sequential()
lstm_model.add(Embedding(input_dim=max_words, output_dim=64, input_length=max_len))  # Embedding layer
lstm_model.add(LSTM(128))
lstm_model.add(Dense(1, activation='sigmoid'))

lstm_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
lstm_history = lstm_model.fit(X_train, y_train, epochs=5, batch_size=128, validation_data=(X_test, y_test))

Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


196/196 ━━━━━━━━━━━━━━━━━━━━ 89s 445ms/step - accuracy: 0.7015 - loss: 0.5444 - val_accuracy: 0.8221 - val_loss: 0.3962
Epoch 2/5
196/196 ━━━━━━━━━━━━━━━━━━━━ 87s 443ms/step - accuracy: 0.8932 - loss: 0.2679 - val_accuracy: 0.8515 - val_loss: 0.3392
Epoch 3/5
196/196 ━━━━━━━━━━━━━━━━━━━━ 143s 446ms/step - accuracy: 0.9235 - loss: 0.2102 - val_accuracy: 0.8431 - val_loss: 0.4159
Epoch 4/5
196/196 ━━━━━━━━━━━━━━━━━━━━ 105s 538ms/step - accuracy: 0.9424 - loss: 0.1595 - val_accuracy: 0.8417 - val_loss: 0.3862
Epoch 5/5
196/196 ━━━━━━━━━━━━━━━━━━━━ 143s 544ms/step - accuracy: 0.9574 - loss: 0.1208 - val_accuracy: 0.8349 - val_loss: 0.5626


In [29]:
gru_eval = gru_model.evaluate(X_test, y_test)
print(f"GRU Model Accuracy: {gru_eval[1] * 100:.2f}%")

lstm_eval = lstm_model.evaluate(X_test, y_test)
print(f"LSTM Model Accuracy: {lstm_eval[1] * 100:.2f}%")

782/782 ━━━━━━━━━━━━━━━━━━━━ 25s 31ms/step - accuracy: 0.8344 - loss: 0.4679
GRU Model Accuracy: 83.59%
782/782 ━━━━━━━━━━━━━━━━━━━━ 44s 56ms/step - accuracy: 0.8348 - loss: 0.5697
LSTM Model Accuracy: 83.49%


In [30]:
if gru_eval[1] > lstm_eval[1]:
    print("The GRU model outperforms the LSTM model.")
elif gru_eval[1] < lstm_eval[1]:
    print("The LSTM model outperforms the GRU model.")
else:
    print("Both models perform equally well.")

The GRU model outperforms the LSTM model.
